## 2. LIMPIEZA

### 1. Importamos las librerías necesarias.

In [109]:
# Herramientas para EDA
import pandas as pd
import numpy as np
import sidetable as stb
from datetime import timedelta
from datetime import date, datetime
import holidays

# Herramientas para la Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors 

### 2. Abrimos el archivo con el que vamos a trabajar.

In [110]:
df = pd.read_csv('../archivos/bikes.csv', index_col=0)
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


### 3. Después de un EDA inicial, procedemos a la limpieza de nuestros datos.

3.1. Eliminación de columnas.

In [111]:
# Eliminamos la columna instant, porqué es redundante

df.drop(['instant'], axis=1, inplace=True)

3.2. Traducción de columnas y valores.

In [112]:
# Renombramos las columnas para que esté toda nuestra info en un mismo idioma
nuevas_columnas= ['fecha', 'estacion', 'año', 'mes', 'dias_festivos', 'dias_de_la_semana', 'dias_laborales', 'clima', 'temperatura', 'sensacion_termica', 'humedad', 'velocidad_viento', 'clientes_casuales', 'clientes_registrados', 'total_bicis']
df.columns = nuevas_columnas

3.3. Corregimos los días de la semana.

In [113]:
for i,v in enumerate(df['fecha']):
    days= ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
    d= datetime.strptime(v, '%d-%m-%Y') #damos la vuelta a la fecha
    df.loc[i,'dia_semana'] = days[d.weekday()] 

In [114]:
df.head()

,fecha,estacion,año,mes,dias_festivos,dias_de_la_semana,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
0,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,lunes
1,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,martes
2,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,miercoles
3,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,jueves
4,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,viernes


In [115]:
df.drop(['dias_de_la_semana'], axis=1, inplace=True)

3.4. Las estaciones estaban intercambiadas, ponemos los nombres correctos.

In [116]:
df['estacion'].unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

In [117]:
dic_est = {'spring' : 'invierno', 'winter' : 'otoño', 'summer' : 'primavera', 'autumn' : 'verano'}
df['estacion'] = df['estacion'].map(dic_est)

In [118]:
df.sample(5)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
178,28-06-2018,verano,0,6,0,1,1,30.510847,34.62790,63.4167,9.666961,732,3916,4648,jueves
393,29-01-2019,invierno,1,1,0,0,1,11.582500,13.63605,31.1250,16.083350,558,2685,3243,martes
390,26-01-2019,invierno,1,1,0,1,2,14.008347,17.80250,76.9583,4.917519,244,3831,4075,sabado
317,14-11-2018,otoño,0,11,0,1,1,21.730000,26.23060,58.7083,20.541932,595,3891,4486,miercoles
212,01-08-2018,verano,0,8,0,1,1,31.638347,35.16460,55.0833,10.500039,729,3537,4266,miercoles


3.5. Corregimos los días festivos.

In [119]:
df['dias_festivos'].value_counts()

0    709
1     21
Name: dias_festivos, dtype: int64

In [120]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%d-%m-%Y")

In [121]:
df['dias_festivos'] = pd.Series(df['fecha']).apply(lambda x: holidays.CountryHoliday('US', state='WA').get(x)).values

In [122]:
df['dias_festivos'].nunique()

11

In [123]:
df['dias_festivos'].value_counts()

New Year's Day                2
Martin Luther King Jr. Day    2
Washington's Birthday         2
Memorial Day                  2
Independence Day              2
Labor Day                     2
Columbus Day                  2
Veterans Day                  2
Thanksgiving                  2
Christmas Day                 2
Veterans Day (Observed)       1
Name: dias_festivos, dtype: int64

In [124]:
df['dias_festivos'] = df['dias_festivos'] #.astype('bool').astype('int')

In [125]:
df[(df['dias_festivos'] == 'Veterans Day') | (df['dias_festivos'] == 'Veterans Day (Observed)')] 
#en 2018 pasan el veterands day al lunes, por lo que tenemos un día más festivo (un día menos laborable)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
314,2018-11-11,otoño,0,11,Veterans Day,0,1,13.290847,15.34085,44.6250,21.083225,440,2928,3368,domingo
315,2018-11-12,otoño,0,11,Veterans Day (Observed),0,1,14.623347,17.89710,55.2917,14.208154,1275,2792,4067,lunes
679,2019-11-11,otoño,1,11,Veterans Day,0,1,17.254153,21.08565,65.9167,8.542500,2290,4562,6852,lunes


In [126]:
df.head(2)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
0,2018-01-01,invierno,0,1,New Year's Day,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,lunes
1,2018-01-02,invierno,0,1,None,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,martes


In [127]:
df['dias_festivos'].replace(np.nan, "No Festivo", inplace=True)

In [128]:
clima_cod = {1:'Soleado', 2:'Nublado', 3:'Lluvia', 4:'Tormenta'}
df['clima'] = df['clima'].map(clima_cod)

In [129]:
df.head()

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
0,2018-01-01,invierno,0,1,New Year's Day,0,Nublado,14.110847,18.18125,80.5833,10.749882,331,654,985,lunes
1,2018-01-02,invierno,0,1,No Festivo,0,Nublado,14.902598,17.68695,69.6087,16.652113,131,670,801,martes
2,2018-01-03,invierno,0,1,No Festivo,1,Soleado,8.050924,9.47025,43.7273,16.636703,120,1229,1349,miercoles
3,2018-01-04,invierno,0,1,No Festivo,1,Soleado,8.200000,10.60610,59.0435,10.739832,108,1454,1562,jueves
4,2018-01-05,invierno,0,1,No Festivo,1,Soleado,9.305237,11.46350,43.6957,12.522300,82,1518,1600,viernes


In [130]:
df.to_csv('../archivos/bikes_limpio.csv')

3.6. Estandarizamos los outliers.

In [131]:
# Ahora nos centramos en nuestros outliers
numericas = df.select_dtypes(include=np.number) 
numericas.head()

,año,mes,dias_laborales,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis
0,0,1,0,14.110847,18.18125,80.5833,10.749882,331,654,985
1,0,1,0,14.902598,17.68695,69.6087,16.652113,131,670,801
2,0,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,0,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,0,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [132]:
lista_columnas = numericas.columns  

In [133]:
def detectar_outliers(lista_columnas, dataframe): 
    
    dicc_indices = {} 
    
    for col in lista_columnas:
        
        Q1 = np.nanpercentile(numericas[col], 25)
        Q3 = np.nanpercentile(numericas[col], 75)
     
        IQR = Q3 - Q1
        
        outlier_step = 1.5 * IQR
    
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        
        
        if outliers_data.shape[0] > 0: 
        
            dicc_indices[col] = (list(outliers_data.index)) 
        
    return dicc_indices 

In [134]:
indices= detectar_outliers(lista_columnas, df)  

In [135]:
indices.keys() 

dict_keys(['humedad', 'velocidad_viento', 'clientes_casuales'])

In [136]:
outliers_humedad = list(indices['humedad'])
outliers_velocidad_viento = list(indices['velocidad_viento'])
outliers_festivos = list(indices['dias_festivos'])
outliers_clientes = list(indices['clientes_casuales'])

KeyError: 'dias_festivos'

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'humedad':
for i in outliers_humedad:
    df.replace(numericas.loc[i,'humedad'], numericas['humedad'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'velocidad_viento':
for i in outliers_velocidad_viento:
    df.replace(numericas.loc[i,'velocidad_viento'], numericas['velocidad_viento'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'dias festivos':
for i in outliers_festivos:
    df.replace(numericas.loc[i,'dias_festivos'], numericas['dias_festivos'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'clientes_casuales':
for i in outliers_clientes:
    df.replace(numericas.loc[i,'clientes_casuales'], numericas['clientes_casuales'].median(), inplace=True)